<a href="https://colab.research.google.com/github/CommonCrayon/Movie-Recommendation/blob/main/Evalutation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile

def extract_zip_file(zip_file_path, extraction_path):
    if os.path.exists(zip_file_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extraction_path)
        print("Extraction successful.")
    else:
        print("Download from:\n" +
              "https://www.dropbox.com/s/ip7x5v26a5kvixg/ml-100k.zip?e=2&file_subpath=%2Fml-100k&dl=0 \n" +
              "https://www.dropbox.com/s/7a1rpq684c33nca/ml-20m.zip?e=2&file_subpath=%2Fml-20m&dl=0 \n"
              )

In [ ]:
extract_zip_file('/content/drive/MyDrive/Movie Recommendation dataset/ml-100k.zip', '/content/drive/MyDrive/Movie Recommendation dataset')
extract_zip_file('/content/drive/MyDrive/Movie Recommendation dataset/ml-20m.zip', '/content/drive/MyDrive/Movie Recommendation dataset')

Extraction successful.
Extraction successful.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix


In [ ]:
movies_dataframe = pd.read_csv("/content/drive/MyDrive/Movie Recommendation dataset/ml-100k/u.item", delimiter="|", encoding="latin1",
                               names=["item id", "title", "release date",
                                      "video release date", "IMDb URL", "unknown",
                                      "Action", "Adventure", "Animation",
                                      "Children's", "Comedy", "Crime",
                                      "Documentary", "Drama", "Fantasy",
                                      "Film-Noir", "Horror", "Musical",
                                      "Mystery", "Romance", "Sci-Fi",
                                      "Thriller", "War", "Western"
                                      ])
movies_dataframe = movies_dataframe.drop(["release date", "video release date", "IMDb URL",
                     "unknown", "Action", "Adventure", "Animation",
                     "Children's", "Comedy", "Crime", "Documentary",
                     "Drama", "Fantasy", "Film-Noir", "Horror", "Musical",
                     "Mystery", "Romance", "Sci-Fi","Thriller", "War", "Western"], axis=1)
movies_dataframe

,item id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [ ]:
userbase1_dataframe = pd.read_csv("/content/drive/MyDrive/Movie Recommendation dataset/ml-100k/u1.base", names=['user id', 'item id', 'rating', 'timestamp'], delimiter="\t")
userbase1_dataframe = userbase1_dataframe.drop(["timestamp"], axis=1)
user_list = userbase1_dataframe['user id'].unique().tolist()
user_list
userbase1_dataframe

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [ ]:
usertest1_dataframe = pd.read_csv("/content/drive/MyDrive/Movie Recommendation dataset/ml-100k/u1.test", names=['user id', 'item id', 'rating', 'timestamp'], delimiter="\t")
usertest1_dataframe = usertest1_dataframe.drop(["timestamp"], axis=1)
usertest1_dataframe

,user id,item id,rating
0,1,6,5
1,1,10,3
2,1,12,5
3,1,14,5
4,1,17,3
...,...,...,...
19995,458,648,4
19996,458,1101,4
19997,459,934,3
19998,460,10,3


In [ ]:
# Convert DataFrame to sparse matrix
user_item_matrix = userbase1_dataframe.pivot(index='user id', columns='item id', values='rating').fillna(0)
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)

# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(user_item_matrix_sparse)
cosine_sim_matrix

array([[1.        , 0.09702087, 0.05246924, ..., 0.15815144, 0.13309031,
        0.25277792],
       [0.09702087, 1.        , 0.05134808, ..., 0.13280638, 0.1265973 ,
        0.10178363],
       [0.05246924, 0.05134808, 1.        , ..., 0.08353707, 0.08122974,
        0.01967584],
       ...,
       [0.15815144, 0.13280638, 0.08353707, ..., 1.        , 0.1016418 ,
        0.09511958],
       [0.13309031, 0.1265973 , 0.08122974, ..., 0.1016418 , 1.        ,
        0.18246466],
       [0.25277792, 0.10178363, 0.01967584, ..., 0.09511958, 0.18246466,
        1.        ]])

In [ ]:
utility_matrix = userbase1_dataframe.pivot(index='user id', columns='item id', values='rating').fillna(0)
utility_matrix

item id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Convert cosine similarity matrix to DataFrame for better visualization
# Where both the rows and columns are labeled with user ids, and the values represent the cosine similarity between corresponding users based on their ratings.
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)
cosine_sim_df

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.097021,0.052469,0.021162,0.193545,0.290114,0.200438,0.097786,0.060791,0.200926,...,0.252187,0.090728,0.216186,0.139478,0.156937,0.095265,0.222958,0.158151,0.133090,0.252778
2,0.097021,1.000000,0.051348,0.084426,0.015516,0.187717,0.074479,0.023262,0.108167,0.078176,...,0.083045,0.298811,0.308608,0.403810,0.282896,0.213233,0.189257,0.132806,0.126597,0.101784
3,0.052469,0.051348,1.000000,0.145660,0.037180,0.084526,0.015718,0.086400,0.000000,0.030909,...,0.041330,0.036956,0.135750,0.063940,0.102082,0.034064,0.133720,0.083537,0.081230,0.019676
4,0.021162,0.084426,0.145660,1.000000,0.017291,0.000000,0.037566,0.046087,0.000000,0.023232,...,0.014238,0.047742,0.127305,0.118947,0.089016,0.039116,0.120521,0.135669,0.125925,0.035586
5,0.193545,0.015516,0.037180,0.017291,1.000000,0.158552,0.170890,0.157488,0.053846,0.089508,...,0.284295,0.074215,0.086345,0.063759,0.126358,0.068145,0.198118,0.142141,0.150000,0.239803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.095265,0.213233,0.034064,0.039116,0.068145,0.099518,0.075901,0.039294,0.000000,0.031693,...,0.066039,0.431154,0.258021,0.226449,0.432666,1.000000,0.087687,0.180029,0.043264,0.144250
940,0.222958,0.189257,0.133720,0.120521,0.198118,0.289962,0.263561,0.094273,0.093677,0.246078,...,0.327153,0.107024,0.187536,0.181317,0.175158,0.087687,1.000000,0.145152,0.261376,0.241028
941,0.158151,0.132806,0.083537,0.135669,0.142141,0.093607,0.017469,0.106551,0.110612,0.014989,...,0.046952,0.203301,0.288318,0.234211,0.313400,0.180029,0.145152,1.000000,0.101642,0.095120


In [ ]:
def rating_prediction_weighted(user_ids, cosine_sim_df, utility_matrix, movies_dataframe, top_n=10):
    all_predicted_ratings = []

    for user_id in user_ids:
        # Get cosine similarity scores for the given user
        user_similarity_scores = cosine_sim_df[user_id]

        # Sort users by similarity scores and get top N similar users (excluding the user itself)
        similar_users = user_similarity_scores.sort_values(ascending=False)[1:top_n+1]

        # Initialize a dictionary to store aggregated ratings from similar users
        aggregated_ratings = {}
        predicted_ratings = {}

        # Aggregate ratings from similar users for items not yet rated by the given user
        for similar_user_id, similarity_score in similar_users.items():
            # Get items rated by the similar user that the given user has not yet rated
            unrated_items = utility_matrix.loc[user_id][utility_matrix.loc[user_id] == 0].index

            # Get ratings of unrated items by the similar user
            similar_user_ratings = utility_matrix.loc[similar_user_id, unrated_items]

            # Aggregate ratings from similar user
            for item_id,rating in similar_user_ratings.items():
              weighted_sum = 0
              sum_of_weights = 0
              if rating != 0:  # Exclude users who haven't rated the item
                    weighted_sum += rating * similarity_score
                    sum_of_weights += similarity_score



              if sum_of_weights != 0:
                predicted_rating = weighted_sum / sum_of_weights
                predicted_ratings[item_id] = predicted_rating

            # Append predicted ratings for the current user to the list
            all_predicted_ratings.extend(
                  [{'user_id': user_id, 'item_id': item_id, 'rating': rating} for item_id, rating in predicted_ratings.items()]
              )

          # Convert the list of dictionaries to a DataFrame
        predicted_ratings_df = pd.DataFrame(all_predicted_ratings)
    return predicted_ratings



In [ ]:
# Evaluation function
def evaluate_predictions(test_df, predicted_ratings):
    # Merge test and predicted DataFrames on user_id and item_id
    merged_df = test_df.merge(pd.DataFrame(predicted_ratings.items(), columns=['item_id', 'predicted_rating']), on='item_id', how='inner')

    # Compute Root Mean Squared Error (RMSE)
    rmse = np.sqrt(((merged_df['rating'] - merged_df['predicted_rating']) ** 2).mean())

    return rmse

In [ ]:
def evaluate_predictions(user_ids, cosine_sim_df, utility_matrix, test_df):
    all_predicted_ratings = {}

    for user_id in user_ids:
        # Get cosine similarity scores for the given user
        user_similarity_scores = cosine_sim_df[user_id]

        # Find unrated items for the given user in the test dataset
        unrated_items = test_df[test_df['user_id'] == user_id]['item_id'].values

        # Initialize a dictionary to store predicted ratings for unrated items
        predicted_ratings = {}

        # Predict ratings for unrated items
        for item_id in unrated_items:
            # Aggregate ratings from similar users
            weighted_sum = 0
            sum_of_weights = 0
            for similar_user_id, similarity_score in user_similarity_scores.iteritems():
                # Get the rating of the similar user for the item
                rating = utility_matrix.loc[similar_user_id, item_id]
                if rating != 0:  # Exclude users who haven't rated the item
                    weighted_sum += rating * similarity_score
                    sum_of_weights += similarity_score

            if sum_of_weights != 0:
                predicted_rating = weighted_sum / sum_of_weights
                predicted_ratings[item_id] = predicted_rating

        all_predicted_ratings[user_id] = predicted_ratings

    return all_predicted_ratings

In [ ]:
users_id = user_list
predict_items_df = rating_prediction_weighted(users_id, cosine_sim_df, utility_matrix, movies_dataframe,50)
predict_items_df
test_df = usertest1_dataframe

In [ ]:
def evaluate_mse(predicted_ratings_df, test_df):
    # Merge predicted ratings with test DataFrame on user_id and item_id
    merged_df = pd.merge(test_df, predicted_ratings_df, on=['user_id', 'item_id'], how='inner')

    # Compute squared error between predicted and actual ratings
    merged_df['squared_error'] = (merged_df['rating'] - merged_df['predicted_rating']) ** 2

    # Calculate mean squared error (MSE)
    mse = merged_df['squared_error'].mean()

    return mse

# Example usage
mse = evaluate_mse(predict_items_df, test_df)
print("Mean Squared Error (MSE):", mse)

In [ ]:
predict_items_df

{1: 3.0,
 3: 1.0,
 4: 2.9999999999999996,
 5: 3.0,
 7: 5.0,
 8: 4.0,
 15: 4.0,
 17: 4.0,
 21: 2.0,
 25: 2.0,
 26: 2.0,
 29: 4.0,
 33: 5.0,
 39: 3.0,
 47: 3.0,
 48: 3.0,
 49: 3.0,
 65: 4.0,
 66: 3.0,
 70: 3.0,
 71: 4.0,
 75: 4.0,
 77: 4.0,
 81: 3.0,
 82: 4.0,
 83: 5.0,
 85: 2.9999999999999996,
 86: 4.0,
 87: 4.0,
 88: 2.0,
 89: 4.0,
 95: 4.0,
 108: 2.9999999999999996,
 109: 5.0,
 125: 3.0,
 128: 5.0,
 135: 4.0,
 143: 2.9999999999999996,
 144: 4.0,
 147: 5.0,
 148: 4.0,
 150: 4.0,
 153: 4.0,
 154: 4.0,
 156: 5.0,
 157: 4.0,
 158: 2.0,
 159: 3.0,
 163: 3.0,
 164: 2.9999999999999996,
 167: 4.0,
 175: 4.0,
 176: 4.0,
 179: 3.0,
 180: 5.0,
 183: 4.0,
 190: 4.0,
 191: 5.0,
 192: 2.9999999999999996,
 209: 5.0,
 211: 4.0,
 222: 5.0,
 223: 4.0,
 235: 2.0,
 238: 5.0,
 240: 2.0,
 241: 4.0,
 243: 4.0,
 245: 2.0,
 246: 3.0,
 248: 4.0,
 249: 5.0,
 250: 4.0,
 252: 1.0,
 254: 2.9999999999999996,
 255: 2.0,
 257: 3.0,
 258: 2.9999999999999996,
 259: 2.0,
 263: 1.0,
 265: 4.0,
 268: 3.0,
 271: 4.0,
 272:

In [ ]:
rsme = evaluate_predictions(test_df, predict_items_df)

KeyError: 'item_id'

In [ ]:
def evaluation_prediction(test_df, ):